In [1]:
import tensorflow as tf
import keras
import pandas as pd
import numpy as np
import sklearn.model_selection

In [2]:
# Code based on a ChatGPT answer

def dataframe_to_series(df):
  """ Convert a specially-formatted dataframe to a rank-3 NumPy array
  of shape (num_series, num_steps, num_channels).
  """
  
  df_sorted = df.sort_values(by=['Time_Series_ID', 'Time_Step'])
  
  # Extract the number of time series and number of time steps
  num_series = df['Time_Series_ID'].nunique()
  num_steps  = df['Time_Step'].nunique()
  
  # Extract the values corresponding to the channels
  # Assuming the channel columns are named as 'Channel_0', 'Channel_1', etc.
  channel_columns = [col for col in df if col.startswith('Channel_')]
  channel_values = df_sorted[channel_columns].values
  
  # Reshape back to 3D array
  num_channels = len(channel_columns)
  
  original_shape = (num_series, num_steps, num_channels)
  X = channel_values.reshape(original_shape)
  
  return X

def make_eeg_data():
  data_dir = "../../datasets/fake_brain_data/"
  df0 = pd.read_csv(data_dir+'eeg_class0.csv')
  df1 = pd.read_csv(data_dir+'eeg_class1.csv')
  
  X0 = dataframe_to_series(df0)[:500]
  X1 = dataframe_to_series(df1)[:500]
  y0 = np.zeros(X0.shape[0])
  y1 = np.ones(X1.shape[0])
  
  X = np.concatenate([X0, X1])
  y = np.concatenate([y0, y1])
  
  return X,y

In [3]:
X, y = make_eeg_data()
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y,
  test_size=0.3, random_state=0)


FileNotFoundError: [Errno 2] No such file or directory: '../../datasets/fake_brain_data/eeg_class0.csv'

In [ ]:
def get_mi_model():
  inputs = keras.layers.Input(shape=X_train.shape[1:])
  x = keras.layers.SimpleRNN(1)(inputs)   # 1 recurrent neuron
  x = keras.layers.Dense(1, activation='sigmoid')(x)
  model = keras.models.Model(inputs, x)
  return model
get_mi_model().summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 48, 3)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_2 (SimpleRNN)        │ (None, 1)              │             5 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │             2 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7 (28.00 B)

 Trainable params: 7 (28.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
input = keras.layers.Input((None, 1))
x = keras.layers.GRU(4, return_sequences=True)(input)
x = keras.layers.GRU(1)(x)
x = keras.layers.Dense(1, activation='softmax')(x)
model = keras.Model(input, x)
model.summary()


Model: "functional_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_13 (InputLayer)     │ (None, None, 1)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_14 (GRU)                    │ (None, None, 4)        │            84 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_15 (GRU)                    │ (None, 1)              │            21 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 1)              │             2 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 107 (428.00 B)

 Trainable params: 107 (428.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
input = keras.layers.Input((None, 1))
x = keras.layers.SimpleRNN(4, return_sequences=True)(input)
x = keras.layers.LSTM(1)(x)
x = keras.layers.Dense(1, activation='softmax')(x)
model = keras.Model(input, x)
model.summary()


Model: "functional_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_15 (InputLayer)     │ (None, None, 1)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_3 (SimpleRNN)        │ (None, None, 4)        │            24 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_8 (LSTM)                   │ (None, 1)              │            24 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 1)              │             2 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 50 (200.00 B)

 Trainable params: 50 (200.00 B)

 Non-trainable params: 0 (0.00 B)